In [1]:
import os, re
from PIL import Image
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from core.submission import Submission

%matplotlib inline

In [9]:
# load all predictions + images from a folder

model_name = 'vanilla-unet_ext'

image_dir = './../data/test_images/'
prediction_dir = f'./../output/{model_name}/test_output/predictions/'

img = {}
pred = {}

for file in os.listdir(image_dir):
    img_id = int(re.search('\d+', file).group(0))

    img[img_id] = Image.open(image_dir + file)
    pred[img_id] = __to_array(Image.open(prediction_dir + file))

In [10]:
pixel_threshold = 0.5
config = SubmissionConfig('vanilla-unet-ext-0.5')
submission = Submission(config)

for img_id, pred in pred.items():
    # could add postprocessing here
    pred = np.where(pred>pixel_threshold, 1, 0)
    
    submission.add(prediction=pred, img_id=img_id)


    
submission.write()
    

Starting Submission...
Writing Predictions to: ./../output/vanilla-unet-ext-0.5/predictions/
Writing Csv to: ./../output/vanilla-unet-ext-0.5/csv/
Writing Masks to: ./../output/vanilla-unet-ext-0.5/masks/
Writing Overlays to: ./../output/vanilla-unet-ext-0.5/overlays/
Writing Mask Overlays to: ./../output/vanilla-unet-ext-0.5/mask_overlays/
Submission Finished


In [44]:
@widgets.interact
def test(patch_threshold=(0.00,1.00), pixel_threshold=(0.0,1.0)):
    img_id = 204
    #pixel_threshold = None
    patch_threshold = 0.25

    
    overlay = get_mask_overlay(img=img[img_id], prediction=pred[img_id], pixel_threshold=pixel_threshold, patch_threshold= patch_threshold)
    default = get_mask_overlay(img=img[img_id], prediction=pred[img_id], pixel_threshold=None, patch_threshold=0.25)
    
    overlay = np.hstack((overlay, default))
    plt.imshow(np.asarray(overlay))

interactive(children=(FloatSlider(value=0.5, description='patch_threshold', max=1.0), FloatSlider(value=0.5, description='pixel_threshold', max=1.0), Output()), _dom_classes=('widget-interact',))

In [4]:
def get_mask_overlay(img, prediction, pixel_threshold, patch_threshold):
    patch_size = 16
        
    square = {
        0: np.zeros((patch_size,patch_size)),
        1: np.ones((patch_size,patch_size))
    }
        
    mask = np.zeros((prediction.shape[0], prediction.shape[1], 3))

    for j in range(0, prediction.shape[1], patch_size):
        for i in range(0, prediction.shape[0], patch_size):
            patch = prediction[i:i + patch_size, j:j + patch_size]
                    
            if pixel_threshold is not None:
                patch = np.where(patch>pixel_threshold, 1, 0)
                    
            patch_mean = np.mean(patch)
                    
            if patch_mean > patch_threshold:
                label = 1
            else:
                label = 0
                        
            mask[i:i + patch_size, j:j + patch_size, 0] = square[label]
                        
        
    mask = __to_image(mask)
    overlay = __to_overlay(bg=img, overlay=mask)
                    
    return overlay

In [3]:
# utility functions

def __to_overlay(bg, overlay, alpha=0.2):
    bg = bg.convert('RGBA')
    overlay = overlay.convert('RGBA')
    return Image.blend(bg, overlay, alpha)

def __to_image(img_array):
    img_array = (img_array * 255).round().astype(np.uint8)
    return Image.fromarray(img_array)

def __to_array(img):
    img_array = np.asarray(img)
    return img_array / 255.0

In [6]:
class SubmissionConfig():
    
    def __init__(self, model_name):
        self.MODEL_NAME = model_name
        
        self.TEST_OUTPUT_DIR = self.BASE_DIR + 'output/' + self.MODEL_NAME + '/'
    
    BASE_DIR = './../'
    
    
    TEST_PATH_TO_DATA = BASE_DIR + 'data/test_images'
    TEST_IMAGE_SIZE = 608
    TEST_PATCH_FOREGROUND_THRESHOLD = 0.25
    
    SUB_WRITE_PREDICTIONS = True
    SUB_WRITE_CSV = True
    SUB_WRITE_MASKS = True
    SUB_WRITE_OVERLAYS = True
    SUB_WRITE_MASK_OVERLAYS = True
    